In [1]:
!pip install langchain-google-genai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.3.2
    Uninstalling google-generativeai-0.3.2:
      Successfully uninstalled google-generativeai-0.3.2


In [9]:
!pip install PyPDF2 openpyxl python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


In [6]:
!pip install langchain

  Using cached langchain-0.1.14-py3-none-any.whl (812 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl (28 kB)
  Using cached langchain_community-0.0.31-py3-none-any.whl (1.9 MB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)
  Using cached marshmallow-3.21.1-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)


In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

Provide your Google API Key··········


In [86]:
import re
import PyPDF2
import openpyxl
import csv
from docx import Document


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [26]:
input_prompt="""
Hey Act Like a skilled or very experience ATS(Application Tracking System)
with a deep understanding of tech field,software engineering,data science ,data analyst
and big data engineer. Your task is to extract Candidate Name,Email,Phone Numer from resume.
resume:{text}
I want the response in having this structure and must be list
["Candidate Name":"[]","Email":"[]","Phone Numer":"[]"]
"""

In [28]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(input_prompt)

chain = prompt | llm


In [87]:
def extract_text_from_file(filename):
  if filename.endswith(".pdf"):
    with open(filename, "rb") as pdf_file:
      pdf_reader = PyPDF2.PdfReader(pdf_file)
      text = ""
      for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
  elif filename.endswith(".docx"):
    text = ''
    doc = Document(filename)
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
  elif filename.endswith(".txt"):
    with open(filename, "r") as txt_file:
      text = txt_file.read()
  else:
    raise ValueError("Unsupported file format")
  return text


In [22]:
def save_to_csv(contact_info):
    with open("Contact_information.csv", "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Candidate Name", "Phone Number", "Email Address"])
        csv_writer.writerow(contact_info)

In [34]:
def main():
    try:
        filename = input("Enter resume file path: ")
        text = extract_text_from_file(filename)
        res=chain.invoke({"text": text})
        data_list = [pair.split(':')[1].strip(' "') for pair in res.content.strip('[]').split(',')]
        save_to_csv(data_list)
        print("Contact information extracted and saved to Contact_information.csv")
    except Exception as e:
        print("Error occurred:", e)

if __name__ == "__main__":
    main()

Enter resume file path: /content/Resume.pdf
Contact information extracted and saved to Contact_information.csv


In [56]:
def extract_contact_info(text):
    name_regex = r'[A-Z][a-z]+(?: [A-Z][a-z]+)?'
    name = re.findall(name_regex, text, flags=re.IGNORECASE)
    phone_regex = r'\b(?:\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})\b'
    email_regex = r"[\w\.-]+@[\w\.-]+\.\w{2,}"
    phone = re.findall(phone_regex, text)
    email = re.findall(email_regex, text)
    return name, phone, email

In [98]:
def save_to_csv1(contact_info):
    with open("Contact_information.csv", "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Candidate Name", "Phone Number", "Email Address"])
        name = contact_info[0][0] if len(contact_info[0]) > 0 else ""
        phone_number = contact_info[1][0] if len(contact_info[1]) > 0 else ""
        email_address = contact_info[2][0] if len(contact_info[2]) > 0 else ""
        csv_writer.writerow([name, phone_number, email_address])

In [99]:
def main():
    try:
        filename = input("Enter resume file path: ")
        text = extract_text_from_file(filename)
        contact_info = extract_contact_info(text)
        save_to_csv1(contact_info)
        print("Contact information extracted and saved to Contact_information.csv")
    except Exception as e:
        print("Error occurred:", e)

if __name__ == "__main__":
    main()

Enter resume file path: /content/Resume.docx
Contact information extracted and saved to Contact_information.csv
